In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def plot_hyperplane_shift(model_low, model_high, snapshot, return_goal_low, return_goal_high, save_path="hyperplane_shift.png"):
    """
    Compare hyperplanes learned under two different return goals by projecting features into 2D
    and plotting the separating boundaries.
    """
    # Unpack snapshot
    X_feat, y, mu, Sigma, real_mu, real_sigma, _ = snapshot
    X_feat = X_feat.numpy() if torch.is_tensor(X_feat) else X_feat
    y = y.numpy() if torch.is_tensor(y) else y

    # --- 1) PCA projection to 2D for visualization
    pca = PCA(n_components=2)
    X_2d = pca.fit_transform(X_feat)

    # --- 2) Get scores from both models (no MVO, just SVM screen)
    def get_scores(model, return_goal):
        with torch.no_grad():
            X_t = torch.as_tensor(X_feat, dtype=torch.double)
            y_t = torch.as_tensor(y, dtype=torch.double)
            mu_t = torch.as_tensor(mu, dtype=torch.double)
            Sigma_t = torch.as_tensor(Sigma, dtype=torch.double)
            w, mask, hinge, C_svm = model(X_t, y_t, mu_t, Sigma_t, return_goal)
            # Forward gives scores inside model
            # Trick: recompute scores directly
            Xp = model.embed(X_t)
            alpha_y = (mask * y_t).view(-1)  # approximate: mask ~ αy
            w_svm = Xp.t().mv(alpha_y)
            scores = Xp @ w_svm
        return scores.numpy()

    scores_low  = get_scores(model_low, return_goal_low)
    scores_high = get_scores(model_high, return_goal_high)

    # --- 3) Meshgrid for decision boundary in PCA plane
    x_min, x_max = X_2d[:,0].min()-0.5, X_2d[:,0].max()+0.5
    y_min, y_max = X_2d[:,1].min()-0.5, X_2d[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))

    # Project mesh back to original space
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_back = pca.inverse_transform(grid_points)

    def get_grid_scores(model, return_goal):
        with torch.no_grad():
            Xg = torch.as_tensor(grid_back, dtype=torch.double)
            Xg_emb = model.embed(Xg)
            # Approximate scores via linear SVM direction
            # (better: re-run SVM QP, but this is fine for viz)
            X_t = torch.as_tensor(X_feat, dtype=torch.double)
            y_t = torch.as_tensor(y, dtype=torch.double)
            mu_t = torch.as_tensor(mu, dtype=torch.double)
            Sigma_t = torch.as_tensor(Sigma, dtype=torch.double)
            _, mask, _, _ = model(X_t, y_t, mu_t, Sigma_t, return_goal)
            alpha_y = (mask * y_t).view(-1)
            w_svm = model.embed(X_t).t().mv(alpha_y)
            scores = Xg_emb @ w_svm
        return scores.view(xx.shape).numpy()

    Z_low  = get_grid_scores(model_low, return_goal_low)
    Z_high = get_grid_scores(model_high, return_goal_high)

    # --- 4) Plot
    plt.figure(figsize=(8,6))
    plt.scatter(X_2d[:,0], X_2d[:,1], c=y, cmap="bwr", edgecolor="k", alpha=0.7, label="Assets")
    plt.contour(xx, yy, Z_low,  levels=[0], colors="blue", linestyles="--", linewidths=2, label="Low goal boundary")
    plt.contour(xx, yy, Z_high, levels=[0], colors="red",  linestyles="-", linewidths=2, label="High goal boundary")

    plt.title(f"Shift of Hyperplane with Different Return Goals\nLow={return_goal_low:.2%}, High={return_goal_high:.2%}")
    plt.xlabel("PCA component 1")
    plt.ylabel("PCA component 2")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Saved hyperplane shift plot to {save_path}")

In [ ]:
# Suppose you trained two models
from train import train_no_crisis

df = factor_df_prep("feature_data/crisis_data.csv")
snapshots = estimate_returns_covariance(df)
model_low, _, _, _  = train_no_crisis(snapshots, 0.01, 0.05, 0.001, return_goal=0.01, grid_case="low")
model_high, _, _, _ = train_no_crisis(snapshots, 0.01, 0.05, 0.001, return_goal=0.02, grid_case="high")

# Pick a snapshot to visualize
snapshot = results[50]  # arbitrary month

# Plot hyperplane shift
plot_hyperplane_shift(model_low, model_high, snapshot,
                      return_goal_low=0.01,
                      return_goal_high=0.02,
                      save_path="hyperplane_shift.png")